In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from src.web_tasks.single_site_summary import WebsiteContentSummarizer
from rich.markdown import Markdown
from rich.console import Console

ModuleNotFoundError: No module named 'src'

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
openai = OpenAI()
openai_model = "gpt-4o-mini"

In [ ]:
summariser = WebsiteContentSummarizer("https://dirttrackriders.co.uk/about",openai,openai_model)

system_prompt = "You are an assistant that analyses the contents of a website\n \
and provides a comprehensive summary, ignoring text that might be navigation related. \
Respond in markdown."

user_prompt = ("The contents of this website is as follows;\n"
                "please provide a short summary of this website in markdown.\n"
                "If it includes news or announcements, then summarize these too. \n\n"
                "Finish the response with an encouraging message to relevant audience on why they should check out the website.\n" )


In [ ]:
summary = summariser.summarize(system_prompt, user_prompt)
console = Console()
console.print(Markdown(summary))